In [1]:
import datetime
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from chart import *
from document import *

In [2]:
# 메타 정보

today = datetime.date.today()

y = today.year
m = today.month
d = today.day
w = today.weekday()

# report version
type = '월간'
month = '6월'
region = 'KR' # KR|CN|NA|EU|SG|RU
v = '1'

if region == 'KR':
    regex = '^(KR|Global|All)'
elif region == 'CN':
    regex = '^(CN|Global|All)'
elif region == 'NA':
    regex = '^(NA|Global|All)'
elif region == 'EU':
    regex = '^(EU|Global|All)'
elif region == 'SG':
    regex = '^(SG|Global|All)'
elif region == 'RU':
    regex = '^(SG|Global|All)'
else:
    regex = '^(KR|CN|NA|EU|SG|RU|Global|All)'

print("보고서 유형: ", type, "월: ", month)
print(regex)

보고서 유형:  월간 월:  6월
^(KR|Global|All)


## 보고서 표지 (커버 쉬트)

In [3]:
document.add_heading(f'[2022.06] {region} Cloud Infra 운영', level=0)
document.add_heading(f'{y}.{m}.{d}', level=1)
document.add_heading(f'클라우드운영센터', level=1)
document.add_page_break()

## 요약

In [4]:
# Document
document.add_paragraph('요약', style='Intense Quote')

In [5]:
# ETL
base_path = r"./data/요약_202206_서비스관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

# 리전 & 운영계, 월간보고서용
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

In [6]:
# Chart
if region == 'All':
    # Pivoting
    pivot, total = getPivotTable(df)
    source = pivot
    chart = getPieChart(source)
    
    chart.save(f'./charts/summary_{y}_{m}_{d}_{v}_1.png')
    document.add_picture(f'./charts/summary_{y}_{m}_{d}_{v}_1.png')
else:
    pass

In [7]:
# Document
addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.1 모니터링(인시던트핸들링)

In [8]:
# ETL
base_path = r"./data/1_1_202206_모니터링_인시던트핸들링_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

# Pivoting
pivot, total = getPivotTable(df)

# 리전 & 운영계, 월간보고서용
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')] # & (df['월간']=='O')]

# SLA failed
df_failed = df_filtered[ (df_filtered['장애전파'] > 10) & (df_filtered['월']== month)]
df_failed[['날짜', '장애대응조치내용', 'JIRA_Ticket_No', '장애전파']]

# ETLed
#df.head()
#df.info()
#df.describe()

KeyError: '장애전파'

In [ ]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 스캐터 차트    
chart = getScatterChart(timeslot, month)
chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_1.png')

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_2.png')

In [11]:
# Document
document.add_paragraph('모니터링(인시던트핸들링)', style='List Number')

if region == 'All':
    document.add_paragraph(f'6월: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_1.png')
    document.add_paragraph(f'월별 발생 건수 (추세)', style='List Bullet')
    document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_2.png')      
else:
    ## 평균 소요 시간, 최대 소요 시간
    document.add_paragraph(f'6월: 총 {len(timeslot)}', style='List Bullet')
    document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_1.png')
    document.add_paragraph(f'장애전파시간 초과 이벤트', style='List Bullet')
    addFailedTable(df_failed, regex, month)
    document.add_paragraph(f'월별 발생 건수 (추세)', style='List Bullet')
    document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_2.png')   

In [ ]:
# Document
document.add_page_break()
document.save('demo.docx')

### 발생 건수(월별 추세)

In [ ]:
chart

In [ ]:
pTable

In [ ]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [ ]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

## 1.2 이슈 관리

In [11]:
# Data Loading
base_path = r"./data/1_2_202206_이슈관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   년도         98 non-null     float64
 1   월          98 non-null     object 
 2   리전         98 non-null     object 
 3   AZ         98 non-null     float64
 4   테넌트        98 non-null     object 
 5   진행상태       98 non-null     object 
 6   성패         82 non-null     object 
 7   상태         98 non-null     object 
 8   작업_Title   97 non-null     object 
 9   시작일_Date   98 non-null     object 
 10  완료일        78 non-null     object 
 11  배경_및_작업내용  98 non-null     object 
 12  비고         73 non-null     object 
dtypes: float64(2), object(11)
memory usage: 10.1+ KB


In [14]:
df.describe()

,년도,AZ
count,98.0,98.000000
mean,2022.0,1.040816
std,0.0,0.198882
min,2022.0,1.000000
25%,2022.0,1.000000
50%,2022.0,1.000000
75%,2022.0,1.000000
max,2022.0,2.000000


In [16]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

13


,리전,합계,비중
0,EU,6,46.0
1,CN,2,15.0
2,KR,2,15.0
3,NA,2,15.0
4,SG,1,8.0


In [7]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [5]:
pivot = getPivotTable(df)

In [6]:
source = pivot
chart = get_pieChart(source)
chart

alt.LayerChart(...)

In [7]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [9]:
pTable

,리전,합계,비중
0,KR,11,32.0
1,SG,9,26.0
2,EU,7,21.0
3,CN,5,15.0
4,NA,2,6.0


In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [10]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [16]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

,index,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,105,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2078,DB VM 볼륨 이전 ( SSD -> SAS),DB
1,117,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2063,[API G/W] LB SSL 인증서 갱신 요청,로드밸런서
2,114,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2113,Rancher-webhook-tls 인증서갱신,rancher
3,100,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2045,VM 리소스 증설 요청,컴퓨트
4,123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서
5,91,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2309,GOMS LB 멤버 추가 요청,로드밸런서
6,115,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2523,Rancher-webhook-tls 인증서갱신,rancher
7,95,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2322,CCS 유휴 자원 반납 요청,컴퓨트
8,111,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2326,LB 신규 인증서 추가,로드밸런서
9,110,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2451,Pre-filter 적용 GIS(GTC-C) 요청,방화벽


### 시계열 :: BarChart

In [11]:
source=pTable
chart = get_barChart(source)
chart

alt.LayerChart(...)

In [12]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [14]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [15]:
source

,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 인입 제어(맵 업로드) 요청,로드밸런서
1,22년,1월,CN,1센터,PRD,완료,성공,CNCLOUD-1736,[중국 디지털키 2] 신규 LB 생성 요청,로드밸런서
2,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4129,TAS 플랫폼 Router VM 리소스 증설 요청,컴퓨트
3,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4137,TAS 플랫폼 isolation VM 리소스 증설 요청,컴퓨트
4,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 긴급 지원요청,로드밸런서
...,...,...,...,...,...,...,...,...,...,...
118,22년,6월,SG,1센터,PRD,완료,성공,SGCLOU-990,VM 회수 요청,컴퓨트
119,22년,6월,NA,1센터,PRD,완료,성공,NACLOUD-1448,기존 LB에 멤버 추가,로드밸런서
120,22년,6월,KR,1센터,PRD,완료,성공,KRCLOUD-5123,[API G/W] LB URL 분기 요청,로드밸런서
123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서


In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

KR
CN
NA
EU
SG
RU
Global


#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

## 1.3 장애 관리

In [17]:
# Data Loading
base_path = r"./data/1_3_202206_장애관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   년도               19 non-null     object
 1   월                19 non-null     object
 2   리전               19 non-null     object
 3   AZ               19 non-null     object
 4   테넌트              19 non-null     object
 5   진행상태             19 non-null     object
 6   성패               19 non-null     object
 7   Task_Job         18 non-null     object
 8   System_Service   19 non-null     object
 9   Date             19 non-null     object
 10  Engineer         16 non-null     object
 11  Root_Cause       4 non-null      object
 12  Impact_Severity  15 non-null     object
 13  Description      19 non-null     object
 14  Ticket           12 non-null     object
 15  ReviewDated      4 non-null      object
 16  ReviewResult     2 non-null      object
dtypes: object(17)
memory usage: 2.6+ KB


In [19]:
df.describe()

,년도,월,리전,AZ,테넌트,진행상태,성패,Task_Job,System_Service,Date,Engineer,Root_Cause,Impact_Severity,Description,Ticket,ReviewDated,ReviewResult
count,19,19,19,19,19,19,19,18,19,19,16,4,15,19,12,4,2
unique,1,6,4,2,2,2,2,17,18,17,15,4,15,18,11,3,2
top,2022년,6월,KR,1센터,PRD,완료,성공,북미 데이터 센터 쿨러 이상 발생,NA Datacenter,2022-05-12T00:00:00,"정진호, 심명보, 이한새, 채한길, 우전성, Ahmed. 김의현\n/ 벤더사 직원",DB block corruption,순차별 LB 절체로 인한 영향도 없음,NACLOUD 일부 호스트 온도 알람 발생확인\nipmi 확인 시 온도 상승 확인\...,NACLOUD-1263,2022-06-09T00:00:00,self heal 프로세스(호스트 리부팅) 완료
freq,19,5,13,15,16,18,18,2,2,2,2,1,1,2,2,2,1


In [21]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

3


,리전,합계,비중
0,KR,3,100.0


In [7]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [5]:
pivot = getPivotTable(df)

In [6]:
source = pivot
chart = get_pieChart(source)
chart

alt.LayerChart(...)

In [7]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [9]:
pTable

,리전,합계,비중
0,KR,11,32.0
1,SG,9,26.0
2,EU,7,21.0
3,CN,5,15.0
4,NA,2,6.0


In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [10]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [16]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

,index,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,105,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2078,DB VM 볼륨 이전 ( SSD -> SAS),DB
1,117,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2063,[API G/W] LB SSL 인증서 갱신 요청,로드밸런서
2,114,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2113,Rancher-webhook-tls 인증서갱신,rancher
3,100,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2045,VM 리소스 증설 요청,컴퓨트
4,123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서
5,91,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2309,GOMS LB 멤버 추가 요청,로드밸런서
6,115,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2523,Rancher-webhook-tls 인증서갱신,rancher
7,95,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2322,CCS 유휴 자원 반납 요청,컴퓨트
8,111,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2326,LB 신규 인증서 추가,로드밸런서
9,110,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2451,Pre-filter 적용 GIS(GTC-C) 요청,방화벽


### 시계열 :: BarChart

In [11]:
source=pTable
chart = get_barChart(source)
chart

alt.LayerChart(...)

In [12]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [14]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [15]:
source

,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 인입 제어(맵 업로드) 요청,로드밸런서
1,22년,1월,CN,1센터,PRD,완료,성공,CNCLOUD-1736,[중국 디지털키 2] 신규 LB 생성 요청,로드밸런서
2,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4129,TAS 플랫폼 Router VM 리소스 증설 요청,컴퓨트
3,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4137,TAS 플랫폼 isolation VM 리소스 증설 요청,컴퓨트
4,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 긴급 지원요청,로드밸런서
...,...,...,...,...,...,...,...,...,...,...
118,22년,6월,SG,1센터,PRD,완료,성공,SGCLOU-990,VM 회수 요청,컴퓨트
119,22년,6월,NA,1센터,PRD,완료,성공,NACLOUD-1448,기존 LB에 멤버 추가,로드밸런서
120,22년,6월,KR,1센터,PRD,완료,성공,KRCLOUD-5123,[API G/W] LB URL 분기 요청,로드밸런서
123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서


In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

KR
CN
NA
EU
SG
RU
Global


#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

## 1.4 변경 관리

In [22]:
# Data Loading
base_path = r"./data/1_4_202206_변경관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

In [28]:
df

,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 인입 제어(맵 업로드) 요청,로드밸런서
1,22년,1월,CN,1센터,PRD,완료,성공,CNCLOUD-1736,[중국 디지털키 2] 신규 LB 생성 요청,로드밸런서
2,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4129,TAS 플랫폼 Router VM 리소스 증설 요청,컴퓨트
3,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4137,TAS 플랫폼 isolation VM 리소스 증설 요청,컴퓨트
4,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 긴급 지원요청,로드밸런서
...,...,...,...,...,...,...,...,...,...,...
120,22년,6월,KR,1센터,PRD,완료,성공,KRCLOUD-5123,[API G/W] LB URL 분기 요청,로드밸런서
121,22년,6월,KR,1센터,PRD,진행중,진행중,KRCLOUD-5099,[국내 MCP] Redis Master 절체 요청,DB
122,22년,6월,EU,1센터,PRD,진행중,진행중,EUCLOUD-2482,DB 서버 CPU core / RAM 증설 요청,컴퓨트
123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   년도           125 non-null    object
 1   월            125 non-null    object
 2   리전           125 non-null    object
 3   AZ           125 non-null    object
 4   테넌트          125 non-null    object
 5   진행상태         125 non-null    object
 6   성패           125 non-null    object
 7   Ticket       119 non-null    object
 8   Summary      125 non-null    object
 9   Description  125 non-null    object
dtypes: object(10)
memory usage: 9.9+ KB


In [24]:
df.describe()

,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
count,125,125,125,125,125,125,125,119,125,125
unique,1,6,7,2,1,3,3,113,107,11
top,22년,6월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,신규 LB 생성 요청,로드밸런서
freq,125,37,49,124,125,121,120,2,5,55


In [25]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

34


,리전,합계,비중
0,KR,11,32.0
1,SG,9,26.0
2,EU,7,21.0
3,CN,5,15.0
4,NA,2,6.0


In [7]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [5]:
pivot = getPivotTable(df)

In [6]:
source = pivot
chart = get_pieChart(source)
chart

alt.LayerChart(...)

In [7]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [9]:
pTable

,리전,합계,비중
0,KR,11,32.0
1,SG,9,26.0
2,EU,7,21.0
3,CN,5,15.0
4,NA,2,6.0


In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [10]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [16]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

,index,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,105,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2078,DB VM 볼륨 이전 ( SSD -> SAS),DB
1,117,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2063,[API G/W] LB SSL 인증서 갱신 요청,로드밸런서
2,114,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2113,Rancher-webhook-tls 인증서갱신,rancher
3,100,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2045,VM 리소스 증설 요청,컴퓨트
4,123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서
5,91,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2309,GOMS LB 멤버 추가 요청,로드밸런서
6,115,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2523,Rancher-webhook-tls 인증서갱신,rancher
7,95,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2322,CCS 유휴 자원 반납 요청,컴퓨트
8,111,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2326,LB 신규 인증서 추가,로드밸런서
9,110,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2451,Pre-filter 적용 GIS(GTC-C) 요청,방화벽


### 시계열 :: BarChart

In [11]:
source=pTable
chart = get_barChart(source)
chart

alt.LayerChart(...)

In [12]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [14]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [15]:
source

,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 인입 제어(맵 업로드) 요청,로드밸런서
1,22년,1월,CN,1센터,PRD,완료,성공,CNCLOUD-1736,[중국 디지털키 2] 신규 LB 생성 요청,로드밸런서
2,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4129,TAS 플랫폼 Router VM 리소스 증설 요청,컴퓨트
3,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4137,TAS 플랫폼 isolation VM 리소스 증설 요청,컴퓨트
4,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 긴급 지원요청,로드밸런서
...,...,...,...,...,...,...,...,...,...,...
118,22년,6월,SG,1센터,PRD,완료,성공,SGCLOU-990,VM 회수 요청,컴퓨트
119,22년,6월,NA,1센터,PRD,완료,성공,NACLOUD-1448,기존 LB에 멤버 추가,로드밸런서
120,22년,6월,KR,1센터,PRD,완료,성공,KRCLOUD-5123,[API G/W] LB URL 분기 요청,로드밸런서
123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서


In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

KR
CN
NA
EU
SG
RU
Global


#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

## 1.5 요청 관리

In [26]:
# Data Loading
base_path = r"./data/1_5_202206_요청관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

In [27]:
df

,년,월,리전,AZ,테넌트,진행상태,성패,Summary,Ticket
0,22년,1월,CN,1센터,PRD,완료,성공,중국 PRD_CCS DB ZONE VM 배치,CNCLOUD-1303
1,22년,1월,CN,1센터,STG,완료,성공,cncloudstgComC01 - Storage controller Error log,CNCLOUD-1334
2,22년,1월,CN,1센터,STG,완료,성공,cncloudstgcomc04_IPMI and Zabbix Monitoring st...,CNCLOUD-1450
3,22년,1월,CN,1센터,STG,완료,성공,중국 검증계 UCS 유지보수 계약 만료,CNCLOUD-1595
4,22년,1월,CN,1센터,STG,완료,성공,cncloudstgcomc01_RMA 를 위한 Host empty작업(VM live...,CNCLOUD-1688
...,...,...,...,...,...,...,...,...,...
3462,22년,6월,SG,1센터,PRD,완료,성공,[AU] VM 회수 요청,SGCLOUD-990
3463,22년,6월,SG,1센터,PRD,완료,성공,[AU/SG/ID] 방화벽 추가 오픈 요청,SGCLOUD-991
3464,22년,6월,SG,1센터,PRD,완료,성공,방화벽 등록 요청,SGCLOUD-993
3465,22년,6월,SG,1센터,PRD,완료,성공,[SG] VM list 확인,SGCLOUD-995


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3467 entries, 0 to 3466
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   년        3467 non-null   object
 1   월        3467 non-null   object
 2   리전       3467 non-null   object
 3   AZ       3467 non-null   object
 4   테넌트      3467 non-null   object
 5   진행상태     3467 non-null   object
 6   성패       3467 non-null   object
 7   Summary  3467 non-null   object
 8   Ticket   3467 non-null   object
dtypes: object(9)
memory usage: 243.9+ KB


In [29]:
df.describe()

,년,월,리전,AZ,테넌트,진행상태,성패,Summary,Ticket
count,3467,3467,3467,3467,3467,3467,3467,3467,3467
unique,1,6,7,2,3,1,3,2781,2791
top,22년,3월,KR,1센터,PRD,완료,성공,[AU] 방화벽 해제 요청,SGCLOUD-499
freq,3467,884,1097,3435,2667,3467,3358,43,41


In [30]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

416


,리전,합계,비중
0,EU,116,28.0
1,KR,102,25.0
2,SG,84,20.0
3,CN,52,12.0
4,NA,45,11.0
5,RU,15,4.0
6,IN,2,0.0


In [7]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [5]:
pivot = getPivotTable(df)

In [6]:
source = pivot
chart = get_pieChart(source)
chart

alt.LayerChart(...)

In [7]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [9]:
pTable

,리전,합계,비중
0,KR,11,32.0
1,SG,9,26.0
2,EU,7,21.0
3,CN,5,15.0
4,NA,2,6.0


In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [10]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [16]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

,index,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,105,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2078,DB VM 볼륨 이전 ( SSD -> SAS),DB
1,117,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2063,[API G/W] LB SSL 인증서 갱신 요청,로드밸런서
2,114,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2113,Rancher-webhook-tls 인증서갱신,rancher
3,100,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2045,VM 리소스 증설 요청,컴퓨트
4,123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서
5,91,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2309,GOMS LB 멤버 추가 요청,로드밸런서
6,115,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2523,Rancher-webhook-tls 인증서갱신,rancher
7,95,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2322,CCS 유휴 자원 반납 요청,컴퓨트
8,111,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2326,LB 신규 인증서 추가,로드밸런서
9,110,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2451,Pre-filter 적용 GIS(GTC-C) 요청,방화벽


### 시계열 :: BarChart

In [11]:
source=pTable
chart = get_barChart(source)
chart

alt.LayerChart(...)

In [12]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [14]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [15]:
source

,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 인입 제어(맵 업로드) 요청,로드밸런서
1,22년,1월,CN,1센터,PRD,완료,성공,CNCLOUD-1736,[중국 디지털키 2] 신규 LB 생성 요청,로드밸런서
2,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4129,TAS 플랫폼 Router VM 리소스 증설 요청,컴퓨트
3,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4137,TAS 플랫폼 isolation VM 리소스 증설 요청,컴퓨트
4,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 긴급 지원요청,로드밸런서
...,...,...,...,...,...,...,...,...,...,...
118,22년,6월,SG,1센터,PRD,완료,성공,SGCLOU-990,VM 회수 요청,컴퓨트
119,22년,6월,NA,1센터,PRD,완료,성공,NACLOUD-1448,기존 LB에 멤버 추가,로드밸런서
120,22년,6월,KR,1센터,PRD,완료,성공,KRCLOUD-5123,[API G/W] LB URL 분기 요청,로드밸런서
123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서


In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

KR
CN
NA
EU
SG
RU
Global


#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

## 1.6 자산 관리 - VM

In [50]:
# Data Loading
base_path = r"./data/1_6_202206_자산관리_VM_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

In [51]:
df

,년도,월,리전,AZ,테넌트,count
0,22년,1월,KR,1센터,PRD,2117.0
1,22년,1월,KR,1센터,STG,2145.0
2,22년,1월,KR,1센터,DEV,NaN
3,22년,1월,KR,2센터,PRD,NaN
4,22년,1월,KR,2센터,STG,NaN
...,...,...,...,...,...,...
109,22년,6월,RU,1센터,STG,86.0
110,22년,6월,RU,1센터,DEV,NaN
111,22년,6월,SG,1센터,PRD,243.0
112,22년,6월,SG,1센터,STG,236.0


In [52]:
df = df[df['count'] > 0]

In [53]:
df

,년도,월,리전,AZ,테넌트,count
0,22년,1월,KR,1센터,PRD,2117.0
1,22년,1월,KR,1센터,STG,2145.0
6,22년,1월,EU,1센터,PRD,244.0
7,22년,1월,EU,1센터,STG,242.0
8,22년,1월,NA,1센터,PRD,439.0
...,...,...,...,...,...,...
107,22년,6월,CN,1센터,STG,350.0
108,22년,6월,RU,1센터,PRD,78.0
109,22년,6월,RU,1센터,STG,86.0
111,22년,6월,SG,1센터,PRD,243.0


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78 entries, 0 to 112
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   년도      78 non-null     object 
 1   월       78 non-null     object 
 2   리전      78 non-null     object 
 3   AZ      78 non-null     object 
 4   테넌트     78 non-null     object 
 5   count   78 non-null     float64
dtypes: float64(1), object(5)
memory usage: 4.3+ KB


In [55]:
df.describe()

,count
count,78.000000
mean,569.564103
std,707.263183
min,78.000000
25%,199.500000
50%,262.000000
75%,513.500000
max,2929.000000


In [56]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

3756.0


,리전,합계,비중
0,KR,1931.0,51.0
1,NA,581.0,15.0
2,EU,530.0,14.0
3,CN,393.0,10.0
4,SG,243.0,6.0
5,RU,78.0,2.0


In [57]:
pivot

,리전,합계,비중
0,KR,1931.0,51.0
1,NA,581.0,15.0
2,EU,530.0,14.0
3,CN,393.0,10.0
4,SG,243.0,6.0
5,RU,78.0,2.0


In [7]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [5]:
pivot = getPivotTable(df)

In [6]:
source = pivot
chart = get_pieChart(source)
chart

alt.LayerChart(...)

In [7]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [9]:
pTable

,리전,합계,비중
0,KR,11,32.0
1,SG,9,26.0
2,EU,7,21.0
3,CN,5,15.0
4,NA,2,6.0


In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [10]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [16]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

,index,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,105,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2078,DB VM 볼륨 이전 ( SSD -> SAS),DB
1,117,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2063,[API G/W] LB SSL 인증서 갱신 요청,로드밸런서
2,114,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2113,Rancher-webhook-tls 인증서갱신,rancher
3,100,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2045,VM 리소스 증설 요청,컴퓨트
4,123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서
5,91,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2309,GOMS LB 멤버 추가 요청,로드밸런서
6,115,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2523,Rancher-webhook-tls 인증서갱신,rancher
7,95,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2322,CCS 유휴 자원 반납 요청,컴퓨트
8,111,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2326,LB 신규 인증서 추가,로드밸런서
9,110,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2451,Pre-filter 적용 GIS(GTC-C) 요청,방화벽


### 시계열 :: BarChart

In [11]:
source=pTable
chart = get_barChart(source)
chart

alt.LayerChart(...)

In [12]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [14]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [15]:
source

,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 인입 제어(맵 업로드) 요청,로드밸런서
1,22년,1월,CN,1센터,PRD,완료,성공,CNCLOUD-1736,[중국 디지털키 2] 신규 LB 생성 요청,로드밸런서
2,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4129,TAS 플랫폼 Router VM 리소스 증설 요청,컴퓨트
3,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4137,TAS 플랫폼 isolation VM 리소스 증설 요청,컴퓨트
4,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 긴급 지원요청,로드밸런서
...,...,...,...,...,...,...,...,...,...,...
118,22년,6월,SG,1센터,PRD,완료,성공,SGCLOU-990,VM 회수 요청,컴퓨트
119,22년,6월,NA,1센터,PRD,완료,성공,NACLOUD-1448,기존 LB에 멤버 추가,로드밸런서
120,22년,6월,KR,1센터,PRD,완료,성공,KRCLOUD-5123,[API G/W] LB URL 분기 요청,로드밸런서
123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서


In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

KR
CN
NA
EU
SG
RU
Global


#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

## 1.6 자산 관리 - DB

In [4]:
# Data Loading
base_path = r"./data/1_6_202206_자산관리_DB_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

In [5]:
df

,년도,월,리전,AZ,테넌트,DBMS,count
0,22년,1월,KR,1센터,PRD,PostgreSQL,110.0
1,22년,1월,KR,1센터,PRD,Tibero,28.0
2,22년,1월,KR,1센터,PRD,Maria,32.0
3,22년,1월,KR,1센터,PRD,Redis,97.0
4,22년,1월,KR,1센터,PRD,Mongo,84.0
...,...,...,...,...,...,...,...
625,22년,6월,SG,1센터,DEV,PostgreSQL,0.0
626,22년,6월,SG,1센터,DEV,Tibero,0.0
627,22년,6월,SG,1센터,DEV,Maria,0.0
628,22년,6월,SG,1센터,DEV,Redis,0.0


In [7]:
df = pd.DataFrame([df,
                  index=pd.Index('월', 'DBMS'], name='Actual Label:'),
                  columns=pd.MultiIndex.from_product([df['DBMS], names=['Model:', ]))
df.style

SyntaxError: invalid syntax (1354757296.py, line 2)

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630 entries, 0 to 629
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   년도      630 non-null    object 
 1   월       630 non-null    object 
 2   리전      630 non-null    object 
 3   AZ      630 non-null    object 
 4   테넌트     630 non-null    object 
 5   DBMS    630 non-null    object 
 6   count   630 non-null    float64
dtypes: float64(1), object(6)
memory usage: 34.6+ KB


In [61]:
df.describe()

,count
count,630.000000
mean,9.695238
std,19.908035
min,0.000000
25%,0.000000
50%,1.000000
75%,11.750000
max,122.000000


In [62]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

604.0


,리전,합계,비중
0,KR,407.0,67.0
1,SG,57.0,9.0
2,EU,48.0,8.0
3,CN,43.0,7.0
4,NA,41.0,7.0
5,RU,8.0,1.0


In [ ]:
# nj = df.pivot_table(index='ptype', columns='nj', aggfunc='count').ix[:, 'wd']
# wpt = df.pivot_table(index='ptype', columns='wpt', aggfunc='count').ix[:, 'wd']
# wd = df.pivot_table(index='ptype', columns='wd', aggfunc='count').ix[:, 'nj']
# out = pd.concat([nj, wd, wpt], axis=1, keys=['nj', 'wd', 'wpt']).fillna(0)
# out.columns.names = [None, None]

In [7]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [5]:
pivot = getPivotTable(df)

In [6]:
source = pivot
chart = get_pieChart(source)
chart

alt.LayerChart(...)

In [7]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [9]:
pTable

,리전,합계,비중
0,KR,11,32.0
1,SG,9,26.0
2,EU,7,21.0
3,CN,5,15.0
4,NA,2,6.0


In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [10]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [16]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

,index,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,105,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2078,DB VM 볼륨 이전 ( SSD -> SAS),DB
1,117,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2063,[API G/W] LB SSL 인증서 갱신 요청,로드밸런서
2,114,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2113,Rancher-webhook-tls 인증서갱신,rancher
3,100,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2045,VM 리소스 증설 요청,컴퓨트
4,123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서
5,91,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2309,GOMS LB 멤버 추가 요청,로드밸런서
6,115,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2523,Rancher-webhook-tls 인증서갱신,rancher
7,95,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2322,CCS 유휴 자원 반납 요청,컴퓨트
8,111,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2326,LB 신규 인증서 추가,로드밸런서
9,110,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2451,Pre-filter 적용 GIS(GTC-C) 요청,방화벽


### 시계열 :: BarChart

In [11]:
source=pTable
chart = get_barChart(source)
chart

alt.LayerChart(...)

In [12]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [14]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [15]:
source

,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 인입 제어(맵 업로드) 요청,로드밸런서
1,22년,1월,CN,1센터,PRD,완료,성공,CNCLOUD-1736,[중국 디지털키 2] 신규 LB 생성 요청,로드밸런서
2,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4129,TAS 플랫폼 Router VM 리소스 증설 요청,컴퓨트
3,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4137,TAS 플랫폼 isolation VM 리소스 증설 요청,컴퓨트
4,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 긴급 지원요청,로드밸런서
...,...,...,...,...,...,...,...,...,...,...
118,22년,6월,SG,1센터,PRD,완료,성공,SGCLOU-990,VM 회수 요청,컴퓨트
119,22년,6월,NA,1센터,PRD,완료,성공,NACLOUD-1448,기존 LB에 멤버 추가,로드밸런서
120,22년,6월,KR,1센터,PRD,완료,성공,KRCLOUD-5123,[API G/W] LB URL 분기 요청,로드밸런서
123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서


In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

KR
CN
NA
EU
SG
RU
Global


#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

## 1.6 자산 관리 - K8s

In [63]:
import datetime
from chart import *

# Data Loading
base_path = r"./data/1_6_202206_자산관리_k8s_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   년도                   134 non-null    object 
 1   월                    134 non-null    object 
 2   리전                   134 non-null    object 
 3   AZ                   134 non-null    object 
 4   테넌트                  134 non-null    object 
 5   구분                   134 non-null    object 
 6   k8s클러스터명             134 non-null    object 
 7   K8S Version          134 non-null    object 
 8   Node Qty             134 non-null    float64
 9   CPU_할당율_Cores        134 non-null    object 
 10  CPU_사용률_Cores        133 non-null    float64
 11  CPU_사용_백분율           134 non-null    float64
 12  RAM_할당율_GB           133 non-null    float64
 13  RAM_사용률_GB           133 non-null    float64
 14  RAM_사용_백분율           133 non-null    float64
 15  담당팀                  4 non-null      obj

In [66]:
df.describe()

,Node Qty,CPU_사용률_Cores,CPU_사용_백분율,RAM_할당율_GB,RAM_사용률_GB,RAM_사용_백분율
count,134.000000,133.000000,134.000000,133.000000,133.000000,133.000000
mean,6.776119,21.271429,0.264873,212.812030,50.327068,0.138122
std,5.489076,74.629647,0.248679,659.045067,212.844609,0.183956
min,1.000000,0.400000,0.000000,1.500000,0.000000,0.000000
25%,3.000000,1.000000,0.083333,37.700000,0.200000,0.008850
50%,6.000000,3.500000,0.188333,46.700000,3.800000,0.061008
75%,8.000000,11.300000,0.391250,123.000000,16.700000,0.183274
max,38.000000,700.000000,0.980000,5222.400000,1843.200000,0.794212


In [67]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

52


,리전,합계,비중
0,KR,25,48.0
1,CN,8,15.0
2,NA,7,13.0
3,SG,7,13.0
4,EU,5,10.0


In [60]:
pivot

,리전,합계,비중
0,KR,25,48.0
1,CN,8,15.0
2,NA,7,13.0
3,SG,7,13.0
4,EU,5,10.0


In [7]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [5]:
pivot = getPivotTable(df)

In [6]:
source = pivot
chart = get_pieChart(source)
chart

alt.LayerChart(...)

In [7]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [9]:
pTable

,리전,합계,비중
0,KR,11,32.0
1,SG,9,26.0
2,EU,7,21.0
3,CN,5,15.0
4,NA,2,6.0


In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [10]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [16]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

,index,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,105,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2078,DB VM 볼륨 이전 ( SSD -> SAS),DB
1,117,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2063,[API G/W] LB SSL 인증서 갱신 요청,로드밸런서
2,114,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2113,Rancher-webhook-tls 인증서갱신,rancher
3,100,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2045,VM 리소스 증설 요청,컴퓨트
4,123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서
5,91,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2309,GOMS LB 멤버 추가 요청,로드밸런서
6,115,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2523,Rancher-webhook-tls 인증서갱신,rancher
7,95,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2322,CCS 유휴 자원 반납 요청,컴퓨트
8,111,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2326,LB 신규 인증서 추가,로드밸런서
9,110,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2451,Pre-filter 적용 GIS(GTC-C) 요청,방화벽


### 시계열 :: BarChart

In [11]:
source=pTable
chart = get_barChart(source)
chart

alt.LayerChart(...)

In [12]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [14]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [15]:
source

,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 인입 제어(맵 업로드) 요청,로드밸런서
1,22년,1월,CN,1센터,PRD,완료,성공,CNCLOUD-1736,[중국 디지털키 2] 신규 LB 생성 요청,로드밸런서
2,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4129,TAS 플랫폼 Router VM 리소스 증설 요청,컴퓨트
3,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4137,TAS 플랫폼 isolation VM 리소스 증설 요청,컴퓨트
4,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 긴급 지원요청,로드밸런서
...,...,...,...,...,...,...,...,...,...,...
118,22년,6월,SG,1센터,PRD,완료,성공,SGCLOU-990,VM 회수 요청,컴퓨트
119,22년,6월,NA,1센터,PRD,완료,성공,NACLOUD-1448,기존 LB에 멤버 추가,로드밸런서
120,22년,6월,KR,1센터,PRD,완료,성공,KRCLOUD-5123,[API G/W] LB URL 분기 요청,로드밸런서
123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서


In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

KR
CN
NA
EU
SG
RU
Global


#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

## 1.7 용량 관리

In [68]:
import datetime
from chart import *

# Data Loading
base_path = r"./data/1_7_202206_용량관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df

,년도,월,리전,AZ,테넌트,진행상태,성패,자원유형,Ticket,Summary,Description
0,22년,1월,KR,1.0,PRD,완료,-,Host(CPU),-,신운영계 - 자원증설 필요\n\nCPU Util 80% 초과 - 신규 VM 생성 중단,자원 증설 필요
1,22년,1월,KR,1.0,PRD,완료,-,Host(Memory),-,신운영계 - 자원증설 필요\n\nMemory 60% 초과,자원 증설 필요
2,22년,1월,KR,1.0,PRD,완료,-,Storage,-,신운영계 - 자원증설 필요\n\nNVMe Storage Util 60% 근접,자원 증설 필요
3,22년,1월,KR,1.0,PRD,완료,-,Host(CPU),-,구운영계 - 자원부족\n\nCPU Util 75% 초과 - 신규 VM 생성 중단,자원 부족
4,22년,1월,NA,1.0,PRD,완료,-,Storage,-,자원증설 리뷰 필요\n\nNAPRD Util 50% 초과,증설 리뷰 필요
...,...,...,...,...,...,...,...,...,...,...,...
92,22년,6월,KR,1.0,PRD,완료,-,Host(CPU),-,신운영계 - 자원증설 필요\n\nCPU Util 85% 초과,자원 증설 필요
93,22년,6월,KR,1.0,PRD,완료,-,Host(Memory),-,신운영계 - 자원증설 필요\n\nMemory 60% 초과,자원 증설 필요
94,22년,6월,KR,1.0,PRD,완료,-,Storage,-,신운영계 - 자원증설 필요\n\naggr_node01_NVMe_date 80% 초과,자원 증설 필요
95,22년,6월,KR,1.0,PRD,완료,-,Host(Chage),-,"Pivotal - 1,2,3 VM/CPU/Memory >> PAAS 존으로 편입(0...",자원 증설 필요


In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   년도           97 non-null     object 
 1   월            97 non-null     object 
 2   리전           97 non-null     object 
 3   AZ           97 non-null     float64
 4   테넌트          97 non-null     object 
 5   진행상태         97 non-null     object 
 6   성패           86 non-null     object 
 7   자원유형         97 non-null     object 
 8   Ticket       97 non-null     object 
 9   Summary      97 non-null     object 
 10  Description  97 non-null     object 
dtypes: float64(1), object(10)
memory usage: 8.5+ KB


In [70]:
df.describe()

,AZ
count,97.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [71]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

12


,리전,합계,비중
0,KR,4,33.0
1,EU,3,25.0
2,CN,2,17.0
3,NA,2,17.0
4,RU,1,8.0


In [7]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [5]:
pivot = getPivotTable(df)

In [6]:
source = pivot
chart = get_pieChart(source)
chart

alt.LayerChart(...)

In [7]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [9]:
pTable

,리전,합계,비중
0,KR,11,32.0
1,SG,9,26.0
2,EU,7,21.0
3,CN,5,15.0
4,NA,2,6.0


In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [10]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [16]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

,index,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,105,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2078,DB VM 볼륨 이전 ( SSD -> SAS),DB
1,117,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2063,[API G/W] LB SSL 인증서 갱신 요청,로드밸런서
2,114,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2113,Rancher-webhook-tls 인증서갱신,rancher
3,100,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2045,VM 리소스 증설 요청,컴퓨트
4,123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서
5,91,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2309,GOMS LB 멤버 추가 요청,로드밸런서
6,115,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2523,Rancher-webhook-tls 인증서갱신,rancher
7,95,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2322,CCS 유휴 자원 반납 요청,컴퓨트
8,111,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2326,LB 신규 인증서 추가,로드밸런서
9,110,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2451,Pre-filter 적용 GIS(GTC-C) 요청,방화벽


### 시계열 :: BarChart

In [11]:
source=pTable
chart = get_barChart(source)
chart

alt.LayerChart(...)

In [12]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [14]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [15]:
source

,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 인입 제어(맵 업로드) 요청,로드밸런서
1,22년,1월,CN,1센터,PRD,완료,성공,CNCLOUD-1736,[중국 디지털키 2] 신규 LB 생성 요청,로드밸런서
2,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4129,TAS 플랫폼 Router VM 리소스 증설 요청,컴퓨트
3,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4137,TAS 플랫폼 isolation VM 리소스 증설 요청,컴퓨트
4,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 긴급 지원요청,로드밸런서
...,...,...,...,...,...,...,...,...,...,...
118,22년,6월,SG,1센터,PRD,완료,성공,SGCLOU-990,VM 회수 요청,컴퓨트
119,22년,6월,NA,1센터,PRD,완료,성공,NACLOUD-1448,기존 LB에 멤버 추가,로드밸런서
120,22년,6월,KR,1센터,PRD,완료,성공,KRCLOUD-5123,[API G/W] LB URL 분기 요청,로드밸런서
123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서


In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

KR
CN
NA
EU
SG
RU
Global


#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

## 1.8 백업 관리

In [72]:
import datetime
from chart import *

# Data Loading
base_path = r"./data/1_8_202206_백업관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

In [73]:
df

,연도,월,리전,AZ,테넌트,DBType,백업_정상_유무,Instance,IP,백업_주기_full_archive,비고
0,2022년,6월,KR,1센터,PRD,PostgreSQL,O,ccskrevappdatap01,10.11.26.180,주1/매10분,None
1,2022년,6월,KR,1센터,PRD,PostgreSQL,O,ccskrevappdatap02,10.11.26.181,None,standby
2,2022년,6월,KR,1센터,PRD,PostgreSQL,O,ccskrccspdbp01,10.11.60.250,주1/매10분,None
3,2022년,6월,KR,1센터,PRD,PostgreSQL,O,ccskrccspdbp02,10.11.60.55,None,standby
4,2022년,6월,KR,1센터,PRD,PostgreSQL,O,ccskrtrafficdbp01,10.11.60.251,주1/매10분,None
...,...,...,...,...,...,...,...,...,...,...,...
683,2022년,6월,SG,1센터,PRD,Mongo,X,ccspidmongodbp03,10.107.61.83,None,해외리전은 백업 스토리지 구성 후 작업 예정
684,2022년,6월,SG,1센터,PRD,Mongo,X,ccsidbatchp01,10.107.62.180,None,담당자가 설치 형상이 다름
685,2022년,6월,SG,1센터,PRD,Mongo,X,ccsaumongodbp01,10.107.70.46,None,해외리전은 백업 스토리지 구성 후 작업 예정
686,2022년,6월,SG,1센터,PRD,Mongo,X,ccsaumongodbp02,10.107.71.73,None,해외리전은 백업 스토리지 구성 후 작업 예정


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688 entries, 0 to 687
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   연도                  688 non-null    object
 1   월                   688 non-null    object
 2   리전                  688 non-null    object
 3   AZ                  688 non-null    object
 4   테넌트                 688 non-null    object
 5   DBType              688 non-null    object
 6   백업_정상_유무            594 non-null    object
 7   Instance            682 non-null    object
 8   IP                  682 non-null    object
 9   백업_주기_full_archive  149 non-null    object
 10  비고                  334 non-null    object
dtypes: object(11)
memory usage: 59.2+ KB


In [75]:
df.describe()

,연도,월,리전,AZ,테넌트,DBType,백업_정상_유무,Instance,IP,백업_주기_full_archive,비고
count,688,688,688,688,688,688,594,682,682,149,334
unique,1,1,6,2,1,7,2,665,671,14,50
top,2022년,6월,KR,1센터,PRD,Redis,O,ccskrsvchubredisp02,10.107.19.28,주1/매10분,standby
freq,688,688,468,633,688,194,298,2,2,61,66


In [76]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

688


,리전,합계,비중
0,KR,468,68.0
1,SG,57,8.0
2,NA,54,8.0
3,EU,46,7.0
4,CN,45,7.0
5,RU,18,3.0


In [7]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [5]:
pivot = getPivotTable(df)

In [6]:
source = pivot
chart = get_pieChart(source)
chart

alt.LayerChart(...)

In [7]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [9]:
pTable

,리전,합계,비중
0,KR,11,32.0
1,SG,9,26.0
2,EU,7,21.0
3,CN,5,15.0
4,NA,2,6.0


In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [10]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [16]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

,index,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,105,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2078,DB VM 볼륨 이전 ( SSD -> SAS),DB
1,117,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2063,[API G/W] LB SSL 인증서 갱신 요청,로드밸런서
2,114,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2113,Rancher-webhook-tls 인증서갱신,rancher
3,100,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2045,VM 리소스 증설 요청,컴퓨트
4,123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서
5,91,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2309,GOMS LB 멤버 추가 요청,로드밸런서
6,115,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2523,Rancher-webhook-tls 인증서갱신,rancher
7,95,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2322,CCS 유휴 자원 반납 요청,컴퓨트
8,111,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2326,LB 신규 인증서 추가,로드밸런서
9,110,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2451,Pre-filter 적용 GIS(GTC-C) 요청,방화벽


### 시계열 :: BarChart

In [11]:
source=pTable
chart = get_barChart(source)
chart

alt.LayerChart(...)

In [12]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [14]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [15]:
source

,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 인입 제어(맵 업로드) 요청,로드밸런서
1,22년,1월,CN,1센터,PRD,완료,성공,CNCLOUD-1736,[중국 디지털키 2] 신규 LB 생성 요청,로드밸런서
2,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4129,TAS 플랫폼 Router VM 리소스 증설 요청,컴퓨트
3,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4137,TAS 플랫폼 isolation VM 리소스 증설 요청,컴퓨트
4,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 긴급 지원요청,로드밸런서
...,...,...,...,...,...,...,...,...,...,...
118,22년,6월,SG,1센터,PRD,완료,성공,SGCLOU-990,VM 회수 요청,컴퓨트
119,22년,6월,NA,1센터,PRD,완료,성공,NACLOUD-1448,기존 LB에 멤버 추가,로드밸런서
120,22년,6월,KR,1센터,PRD,완료,성공,KRCLOUD-5123,[API G/W] LB URL 분기 요청,로드밸런서
123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서


In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

KR
CN
NA
EU
SG
RU
Global


#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

## 1.9 보안 관리

In [84]:
# Data Loading
base_path = r"./data/1_9_202206_보안관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

In [85]:
df

,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,22년,1월,KR,1센터,DEV,완료,성공,KRCLOUD-4134,EV 플랫폼 인프라 보안 점검 요청의 건,보안점검
1,22년,1월,KR,1센터,STG,완료,성공,KRCLOUD-4134,EV 플랫폼 인프라 보안 점검 요청의 건,보안점검
2,22년,2월,Global,1센터,ALL,완료,성공,KRCLOUD-4377,글로벌 CCS OS 보안 설정 관련 확인 요청의 건,보안점검
3,22년,2월,CN,1센터,STG,완료,성공,CNCLOUD-1801,공신부 점검 지원 요청 드립니다.,보안점검
4,22년,2월,CN,1센터,PRD,완료,성공,CNCLOUD-1801,공신부 점검 지원 요청 드립니다.,보안점검
...,...,...,...,...,...,...,...,...,...,...
63,22년,6월,KR,1센터,STG,진행중,,KRCLOUD-4612,EV 플랫폼 인프라 보안 점검 요청의 건,보안점검
64,22년,6월,CN,1센터,STG,진행중,,CNCLOUD-2010,클라우드 플랫폼(OpenStack에 관련 보안 설정 현황 확인 요청,보안점검
65,22년,6월,CN,1센터,PRD,진행중,,CNCLOUD-2010,클라우드 플랫폼(OpenStack에 관련 보안 설정 현황 확인 요청,보안점검
66,22년,6월,CN,1센터,PRD,진행중,,CNCLOUD-2005,gcscntmsdbp01 10.107.253.48 DB보안 조치,보안점검


In [86]:
df.describe()

,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
count,68,68,68,68,68,68,68,66,68,68
unique,1,6,7,1,4,2,2,38,36,4
top,22년,5월,SG,1센터,PRD,완료,성공,KRCLOUD-4672,AU SSL 인증서 프로토콜 보안적용 요청,보안점검
freq,68,20,24,68,33,55,55,3,4,46


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   년도           68 non-null     object
 1   월            68 non-null     object
 2   리전           68 non-null     object
 3   AZ           68 non-null     object
 4   테넌트          68 non-null     object
 5   진행상태         68 non-null     object
 6   성패           68 non-null     object
 7   Ticket       66 non-null     object
 8   Summary      68 non-null     object
 9   Description  68 non-null     object
dtypes: object(10)
memory usage: 5.4+ KB


In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   년도           68 non-null     object
 1   월            68 non-null     object
 2   리전           68 non-null     object
 3   AZ           68 non-null     object
 4   테넌트          68 non-null     object
 5   진행상태         68 non-null     object
 6   성패           68 non-null     object
 7   Ticket       66 non-null     object
 8   Summary      68 non-null     object
 9   Description  68 non-null     object
dtypes: object(10)
memory usage: 5.4+ KB


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   년도           68 non-null     object
 1   월            68 non-null     object
 2   리전           68 non-null     object
 3   AZ           68 non-null     object
 4   테넌트          68 non-null     object
 5   진행상태         68 non-null     object
 6   성패           68 non-null     object
 7   Ticket       66 non-null     object
 8   Summary      68 non-null     object
 9   Description  68 non-null     object
dtypes: object(10)
memory usage: 5.4+ KB


In [81]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

2


,리전,합계,비중
0,RU,1,50.0
1,SG,1,50.0


In [7]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [5]:
pivot = getPivotTable(df)

In [6]:
source = pivot
chart = get_pieChart(source)
chart

alt.LayerChart(...)

In [7]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [9]:
pTable

,리전,합계,비중
0,KR,11,32.0
1,SG,9,26.0
2,EU,7,21.0
3,CN,5,15.0
4,NA,2,6.0


In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [10]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [16]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

,index,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,105,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2078,DB VM 볼륨 이전 ( SSD -> SAS),DB
1,117,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2063,[API G/W] LB SSL 인증서 갱신 요청,로드밸런서
2,114,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2113,Rancher-webhook-tls 인증서갱신,rancher
3,100,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2045,VM 리소스 증설 요청,컴퓨트
4,123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서
5,91,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2309,GOMS LB 멤버 추가 요청,로드밸런서
6,115,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2523,Rancher-webhook-tls 인증서갱신,rancher
7,95,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2322,CCS 유휴 자원 반납 요청,컴퓨트
8,111,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2326,LB 신규 인증서 추가,로드밸런서
9,110,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2451,Pre-filter 적용 GIS(GTC-C) 요청,방화벽


### 시계열 :: BarChart

In [11]:
source=pTable
chart = get_barChart(source)
chart

alt.LayerChart(...)

In [12]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [14]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [15]:
source

,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 인입 제어(맵 업로드) 요청,로드밸런서
1,22년,1월,CN,1센터,PRD,완료,성공,CNCLOUD-1736,[중국 디지털키 2] 신규 LB 생성 요청,로드밸런서
2,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4129,TAS 플랫폼 Router VM 리소스 증설 요청,컴퓨트
3,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4137,TAS 플랫폼 isolation VM 리소스 증설 요청,컴퓨트
4,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 긴급 지원요청,로드밸런서
...,...,...,...,...,...,...,...,...,...,...
118,22년,6월,SG,1센터,PRD,완료,성공,SGCLOU-990,VM 회수 요청,컴퓨트
119,22년,6월,NA,1센터,PRD,완료,성공,NACLOUD-1448,기존 LB에 멤버 추가,로드밸런서
120,22년,6월,KR,1센터,PRD,완료,성공,KRCLOUD-5123,[API G/W] LB URL 분기 요청,로드밸런서
123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서


In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

KR
CN
NA
EU
SG
RU
Global


#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

## 1.10 하드웨어 정기정검

In [89]:
# Data Loading
base_path = r"./data/1_10_202206_정기점검_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

In [90]:
df

,년도,월,리전,AZ,테넌트,진행상태,성패,분기별,벤더,추가칼럼(from 컨플),데이터분류,분기별.1
0,22년,1월,KR,1센터,PRD,완료,성공,월,Cisco,KR > Cisco : (월별) 월별 점검 내수 CCS 정기점검 cisco(12월...,Cisco,월
1,22년,1월,KR,1센터,PRD,완료,성공,월,A10,KR>A10 : (월별) 월별 점검 내수 CCS 정기점검 A10(12월).pptx,A10,월
2,22년,1월,KR,1센터,PRD,완료,성공,월,NetApp,KR> NetApp : (월별) 월별 내수 CCS 정기점검 netapp(1월).pdf,NetApp,월
3,22년,1월,KR,1센터,PRD,완료,성공,월,F5,KR>F5 : (월별) 월별 점검 krcloud CCS 정기점검 F5(12월).ppt,F5,월
4,22년,1월,KR,1센터,PRD,완료,성공,분기,Dell,KR>Dell : (분기) 분기점검_Dell_내수 점검문서(21년4분기).pdf,Dell,분기
...,...,...,...,...,...,...,...,...,...,...,...,...
152,22년,6월,CN,1센터,PRD,진행,진행중,월,Paloalto,CN>Paloalto : (월별) 중국-PA-5220_2022.4월.docx,Paloalto,월
153,22년,6월,CN,1센터,PRD,완료,성공,분기,F5,CN>Dell : (분기) 분기점검_Dell_중국 점검문서(22년2분기).pdf,F5,분기
154,22년,6월,SG,1센터,PRD,진행,진행중,월,Paloalto,SG>Paloalto : (월별) 월별_점검_싱가폴-PA-5220(22년4월).docx,Paloalto,월
155,22년,6월,SG,1센터,PRD,완료,성공,월,F5,SG>F5 : (월별) F5 현대오토에버 6월 정기점검(싱가폴_i5800).ppt,F5,월


In [91]:
df.describe()

,년도,월,리전,AZ,테넌트,진행상태,성패,분기별,벤더,추가칼럼(from 컨플),데이터분류,분기별.1
count,157,157,157,157,157,157,157,157,157,157,157,157
unique,1,6,7,2,1,2,2,2,13,148,18,2
top,22년,6월,KR,1센터,PRD,완료,성공,월,NetApp,CN>NetApp : (월별) 월별 점검 중국_netapp(4월).pdf,NetApp,월
freq,157,40,46,156,157,141,141,109,34,2,30,109


In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   년도             157 non-null    object
 1   월              157 non-null    object
 2   리전             157 non-null    object
 3   AZ             157 non-null    object
 4   테넌트            157 non-null    object
 5   진행상태           157 non-null    object
 6   성패             157 non-null    object
 7   분기별            157 non-null    object
 8   벤더             157 non-null    object
 9   추가칼럼(from 컨플)  157 non-null    object
 10  데이터분류          157 non-null    object
 11  분기별.1          157 non-null    object
dtypes: object(12)
memory usage: 14.8+ KB


In [93]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

24


,리전,합계,비중
0,KR,10,42.0
1,CN,3,12.0
2,EU,3,12.0
3,IN,3,12.0
4,NA,3,12.0
5,RU,1,4.0
6,SG,1,4.0


In [7]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [5]:
pivot = getPivotTable(df)

In [6]:
source = pivot
chart = get_pieChart(source)
chart

alt.LayerChart(...)

In [7]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [9]:
pTable

,리전,합계,비중
0,KR,11,32.0
1,SG,9,26.0
2,EU,7,21.0
3,CN,5,15.0
4,NA,2,6.0


In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [10]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [16]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

,index,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,105,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2078,DB VM 볼륨 이전 ( SSD -> SAS),DB
1,117,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2063,[API G/W] LB SSL 인증서 갱신 요청,로드밸런서
2,114,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2113,Rancher-webhook-tls 인증서갱신,rancher
3,100,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2045,VM 리소스 증설 요청,컴퓨트
4,123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서
5,91,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2309,GOMS LB 멤버 추가 요청,로드밸런서
6,115,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2523,Rancher-webhook-tls 인증서갱신,rancher
7,95,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2322,CCS 유휴 자원 반납 요청,컴퓨트
8,111,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2326,LB 신규 인증서 추가,로드밸런서
9,110,22년,6월,EU,1센터,PRD,완료,성공,EUCLOUD-2451,Pre-filter 적용 GIS(GTC-C) 요청,방화벽


### 시계열 :: BarChart

In [11]:
source=pTable
chart = get_barChart(source)
chart

alt.LayerChart(...)

In [12]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [13]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [14]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [15]:
source

,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,Summary,Description
0,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 인입 제어(맵 업로드) 요청,로드밸런서
1,22년,1월,CN,1센터,PRD,완료,성공,CNCLOUD-1736,[중국 디지털키 2] 신규 LB 생성 요청,로드밸런서
2,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4129,TAS 플랫폼 Router VM 리소스 증설 요청,컴퓨트
3,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4137,TAS 플랫폼 isolation VM 리소스 증설 요청,컴퓨트
4,22년,1월,KR,1센터,PRD,완료,성공,KRCLOUD-4125,GIS LB 긴급 지원요청,로드밸런서
...,...,...,...,...,...,...,...,...,...,...
118,22년,6월,SG,1센터,PRD,완료,성공,SGCLOU-990,VM 회수 요청,컴퓨트
119,22년,6월,NA,1센터,PRD,완료,성공,NACLOUD-1448,기존 LB에 멤버 추가,로드밸런서
120,22년,6월,KR,1센터,PRD,완료,성공,KRCLOUD-5123,[API G/W] LB URL 분기 요청,로드밸런서
123,22년,6월,CN,1센터,PRD,완료,성공,CNCLOUD-2103,LB 인증서 갱신 요청,로드밸런서


In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

KR
CN
NA
EU
SG
RU
Global


#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

alt.LayerChart(...)

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

# Save Document

In [ ]:
document.save('word.docx')